In [ ]:
!pip install keras==3.3.3

In [18]:
import os, cv2
import tensorflow as tf
# from model.rep_efficientnet_v2 import EfficientNetV2B3
import keras
# import tensorflow_addons as tfa
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
# from model.losses import categorical_focal_crossentropy
from keras.utils import image_dataset_from_directory
import tensorflow as tf
import glob
import keras
import numpy as np
from keras import Sequential, Model
from keras.layers import Input, Layer, AveragePooling2D, Conv2D, BatchNormalization, Dropout, Dense, Flatten, ReLU, ZeroPadding2D
from tqdm import tqdm


def unfreeze_model(model, unfree=False, precent=10):
    length = len(model.layers)
    top_layer = int(length * (precent/100))
    print("length : ", length, "top_layer: ", top_layer, "int(precent/100): ", precent/100)
    if unfree:
        # We unfreeze the top 20 layers while leaving BatchNorm layers frozen
        top_layer *= -1
        for layer in model.layers[top_layer:]:
            if not isinstance(layer, keras.layers.BatchNormalization):
                layer.trainable = True
                print("Layer >> ", layer)
    return model


def load_images(dir_images, gray=False, dsize=(224, 224)):
    tasks = ('train', 'val')
    data, labels, tmp_paths = [], [],[]
    for _, task in tqdm(enumerate(tasks)):
        cates = os.listdir(os.path.join(dir_images, task))
        for cate in cates:
            files = os.listdir(os.path.join(dir_images, task, cate))
            for img_file in files:
                labels.append(cate)
                tmp_path = os.path.join(dir_images, task, cate, img_file);
                tmp_paths.append(os.path.join(task,cate,img_file))
                image = cv2.imread(tmp_path)
                image = cv2.resize(image, dsize)
                if gray:
                    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
                    data.append(gray_image)
                else:
                    data.append(image)
    return data, labels,tmp_paths


def scale_up_lora(inputs, units, lora_rank, lora_num_layer, activation=None, name='lora'):
    lora_denses = []
    for i in range(lora_num_layer):
        lora_denses.append(keras.layers.Dense(units, activation=activation,
                                              lora_rank=lora_rank, name=f'{name}_dense_{i}')(inputs))
    lora_denses.append(inputs)
    x_nlora = keras.layers.Add(name=f'{name}_add')(lora_denses)
    # x_activation = keras.activations.relu(x_nlora)
    x_bn = keras.layers.BatchNormalization()(x_nlora)
    return x_bn


def build_model(size, base_model):
    inputs = keras.Input(shape=(*size, 3))
    # x_rescaling = keras.layers.Rescaling(scale=1./127.5, offset=-1)(inputs)
    x_base_model = base_model(inputs, training=False)
    x_flatten = keras.layers.GlobalAveragePooling2D()(x_base_model)
    x_dropout1 = keras.layers.Dropout(0.3)(x_flatten)
    # x_activation = keras.activations.relu(x_dropout1)
    x_nlora = scale_up_lora(x_dropout1, units=x_dropout1.shape[1], lora_rank=6, lora_num_layer=4, activation='relu')
    # add_dense = keras.layers.Dense(1792,activation='relu')(x_dropout1)
    x_dropout2 = keras.layers.Dropout(0.3)(x_nlora)
    outputs = keras.layers.Dense(10, activation='softmax')(x_dropout2)
    model = keras.Model(inputs, outputs)
    # init_weights = model.get_weights()
    return model


In [19]:
import os
import shutil


def save_training(save_dir):
    os.makedirs(save_dir, exist_ok=True)
    length = len(glob.glob(save_dir + "/train*"))
    save_dir = os.path.join(save_dir, f'train{length+1}')
    os.makedirs(save_dir, exist_ok=True)
    print(">>>>>>>>>> ", save_dir)
    return save_dir


def get_cacll_back(save_dir):
    checkpoint_filepath = os.path.join(save_dir, 'best.weights.h5')
    model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
        filepath=checkpoint_filepath,
        save_weights_only=True,
        monitor='val_categorical_accuracy',
        mode='max',
        save_best_only=True)

    model_stop = tf.keras.callbacks.EarlyStopping(monitor='val_categorical_accuracy',
                                            verbose=1,
                                            patience=10,
                                            mode='max',
                                            restore_best_weights=False),
    return [
        model_stop,
        model_checkpoint_callback
    ]

In [20]:
base_model = keras.applications.ResNet50(
    weights="imagenet",  # Load weights pre-trained on ImageNet.
    include_top=False,
)

# Freeze the base_model
base_model.trainable = False

# base_model = unfreeze_model(base_model, unfree=True, precent=30) # unfree 30% top layer
# base_model.summary(show_trainable=True)

In [21]:
# temp model
model = build_model(base_model=base_model,size=(224, 224))
init_weights = model.get_weights()
model.summary()


Model: "functional_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_7             │ (None, 224, 224, 3)    │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ resnet50 (Functional)     │ (None, 7, 7, 2048)     │     23,587,712 │ input_layer_7[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ global_average_pooling2d… │ (None, 2048)           │              0 │ resnet50[0][0]         │
│ (GlobalAveragePooling2D)  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_8 (Dropout)       │ (None, 2048)           │              0 │ global_average_poolin… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lora_dense_0 (Dense)      │ (None, 2048)           │      4,220,928 │ dropout_8[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lora_dense_1 (Dense)      │ (None, 2048)           │      4,220,928 │ dropout_8[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lora_dense_2 (Dense)      │ (None, 2048)           │      4,220,928 │ dropout_8[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lora_dense_3 (Dense)      │ (None, 2048)           │      4,220,928 │ dropout_8[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lora_add (Add)            │ (None, 2048)           │              0 │ lora_dense_0[0][0],    │
│                           │                        │                │ lora_dense_1[0][0],    │
│                           │                        │                │ lora_dense_2[0][0],    │
│                           │                        │                │ lora_dense_3[0][0],    │
│                           │                        │                │ dropout_8[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_4     │ (None, 2048)           │          8,192 │ lora_add[0][0]         │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_9 (Dropout)       │ (None, 2048)           │              0 │ batch_normalization_4… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_4 (Dense)           │ (None, 10)             │         20,490 │ dropout_9[0][0]        │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 40,500,106 (154.50 MB)

 Trainable params: 131,082 (512.04 KB)

 Non-trainable params: 40,369,024 (154.00 MB)

In [ ]:
# images, labels = load_images(dir_images="/content/drive/MyDrive/NCKH/HOVA10", gray=False, dsize=(224, 224))
# images = np.array(images)
# labels = np.array(labels)
# images.shape, labels.shape
# from sklearn import preprocessing
# from sklearn.model_selection import StratifiedKFold
# # setup KFOLD
# lb = preprocessing.LabelBinarizer()
# skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=10000)
# skf.get_n_splits(images, labels)
# print(skf)

# # setup model
# model = build_model(base_model=base_model,size=(224, 224))
# init_weights = model.get_weights()

2it [18:41, 560.99s/it]


StratifiedKFold(n_splits=5, random_state=10000, shuffle=True)


In [22]:
model = build_model(base_model=base_model,size=(224, 224))
init_weights = model.get_weights()

In [ ]:
save_kfold = "/content/drive/MyDrive/DATASET/logs/hova10"
save_kfold_path = save_training(save_kfold)
save_kfold_path

In [24]:
def load_images(dir_images, gray=False, dsize=(224, 224)):
    data  = []
    for dir_image in dir_images:
      image = cv2.imread(dir_image)
      image = cv2.resize(image, dsize)
      if gray:
        gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        data.append(gray_image)
      else:
        data.append(image)
    return data

In [ ]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
dir_image = "/content/drive/MyDrive/NCKH/HOVA10"
save_path = "/content/drive/MyDrive/NCKH/Data_Folds"
lb = preprocessing.LabelBinarizer()
batch_size = 32
epoch = 100
for i in range(5):
  train_csv = pd.read_csv(os.path.join(save_path, f"KFold{i}", "train_path.csv"))
  test_csv = pd.read_csv(os.path.join(save_path, f"KFold{i}", "test_path.csv"))

  train_img_path = dir_image + "/" + train_csv['path'].astype(str).to_numpy()
  data_train = load_images(train_img_path)
  data_train = np.array(data_train)
  label_train = train_csv['label'].to_numpy()
  labels_one_hot_train = lb.fit_transform(label_train)


  test_img_path = dir_image + "/" + test_csv['path'].astype(str).to_numpy()
  data_test = load_images(test_img_path)
  data_test = np.array(data_test)
  label_test = test_csv['label'].to_numpy()
  labels_one_hot_test = lb.fit_transform(label_test)
  print(f"Fold {i}:")
  print("train : ", data_train.shape, labels_one_hot_train.shape)
  print("val : ", data_test.shape, labels_one_hot_test.shape)
  focal = keras.losses.categorical_focal_crossentropy
  METRICS = [
        # tf.keras.metrics.TopKCategoricalAccuracy(k=1, name='Top1'),
        tf.keras.metrics.CategoricalAccuracy(name='categorical_accuracy')
    ]
  model.compile(optimizer=tf.keras.optimizers.AdamW(learning_rate=1e-4), loss=[focal], metrics = METRICS)
  my_callbacks = get_cacll_back(os.path.join(save_kfold_path, f"KFold{i}"))
  model.set_weights(init_weights)
  history = model.fit(data_train, labels_one_hot_train,
            epochs = epoch,
            validation_data = (data_test, labels_one_hot_test),
            batch_size=batch_size,
            shuffle=True,
            callbacks=my_callbacks,
            # steps_per_epoch = len(train.file_paths) // batch_size,
            # validation_steps=len(val.file_paths) // batch_size,
            # sample_weight=train.sample_weight
            # workers=5
            #  class_weight = class_weight
    )
  model.save_weights(os.path.join(save_kfold_path, f"KFold{i}", 'last.weights.h5'))
  print("save to >>>>>>>>>> ", os.path.join(save_kfold_path, f"KFold{i}", 'last.weights.h5'))

  import pandas as pd

    # convert the history.history dict to a pandas DataFrame:
  hist_df = pd.DataFrame(history.history)
  hist_csv_file = os.path.join(save_kfold_path, f"KFold{i}", 'history.csv')
  with open(hist_csv_file, mode='w') as f:
        hist_df.to_csv(f, index=False)

  model.load_weights(os.path.join(save_kfold_path, f"KFold{i}", "best.weights.h5"))
  val_metrics = model.evaluate(data_test, labels_one_hot_test, batch_size=batch_size)
  print("Val Loss: ", val_metrics[0])
  print("Val Acc: ", val_metrics[1])




